In [1]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover

In [2]:
appName = "Amazon alexa Review System"
spark = SparkSession \
    .builder \
    .appName(appName) \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [3]:
Amazon_csv = spark.read.csv('output.csv', inferSchema=True, header=True)
Amazon_csv.show(truncate=True, n=3)

+-------+---------+----------------+--------------------+--------+
| rating|     date|       variation|    verified_reviews|feedback|
+-------+---------+----------------+--------------------+--------+
|      5|31-Jul-18|Charcoal Fabric |       Love my Echo!|       1|
|      5|31-Jul-18|Charcoal Fabric |           Loved it!|       1|
|      4|31-Jul-18|  Walnut Finish |Sometimes while p...|       1|
+-------+---------+----------------+--------------------+--------+
only showing top 3 rows



In [4]:
print((Amazon_csv.count(), len(Amazon_csv.columns)))

(3150, 5)


In [5]:
Amazon_csv.select('feedback').distinct().collect()

[Row(feedback=' this would be a great gift. Once connected to wifi'),
 Row(feedback='0'),
 Row(feedback=' but getting the hang of it."'),
 Row(feedback=' it\'s pretty creepy"'),
 Row(feedback=' but hopefully soon Amazon allows the option to shut it off."'),
 Row(feedback=' gives you the wrong answer or finds information off of Wikipedia!"'),
 Row(feedback='1'),
 Row(feedback=' I can control my lights'),
 Row(feedback='  but I\'ll probably never use it"". Was I wrong!!  Echo is part of my daily routine')]

In [6]:
Amazon_csv = Amazon_csv.filter((Amazon_csv.feedback==1) | (Amazon_csv.feedback==0))
Amazon_csv.show(truncate=True, n=10)

+-------+---------+--------------------+--------------------+--------+
| rating|     date|           variation|    verified_reviews|feedback|
+-------+---------+--------------------+--------------------+--------+
|      5|31-Jul-18|    Charcoal Fabric |       Love my Echo!|       1|
|      5|31-Jul-18|    Charcoal Fabric |           Loved it!|       1|
|      4|31-Jul-18|      Walnut Finish |Sometimes while p...|       1|
|      5|31-Jul-18|    Charcoal Fabric |I have had a lot ...|       1|
|      5|31-Jul-18|    Charcoal Fabric |               Music|       1|
|      5|31-Jul-18|Heather Gray Fabric |I received the ec...|       1|
|      3|31-Jul-18|   Sandstone Fabric |Without having a ...|       1|
|      5|31-Jul-18|    Charcoal Fabric |I think this is t...|       1|
|      5|30-Jul-18|Heather Gray Fabric |         looks great|       1|
|      5|30-Jul-18|Heather Gray Fabric |Love it! I’ve lis...|       1|
+-------+---------+--------------------+--------------------+--------+
only s

In [7]:
Amazon_csv.select('feedback').distinct().collect()

[Row(feedback='0'), Row(feedback='1')]

In [8]:
Amazon_csv_ = Amazon_csv.filter(col("verified_reviews").isNotNull()).select( "verified_reviews", "feedback")
Amazon_csv_.show(truncate=True, n=10)

+--------------------+--------+
|    verified_reviews|feedback|
+--------------------+--------+
|       Love my Echo!|       1|
|           Loved it!|       1|
|Sometimes while p...|       1|
|I have had a lot ...|       1|
|               Music|       1|
|I received the ec...|       1|
|Without having a ...|       1|
|I think this is t...|       1|
|         looks great|       1|
|Love it! I’ve lis...|       1|
+--------------------+--------+
only showing top 10 rows



In [9]:
Amazon_csv_.filter(Amazon_csv.verified_reviews==' ').show()

+----------------+--------+
|verified_reviews|feedback|
+----------------+--------+
|                |       1|
|                |       1|
|                |       1|
|                |       0|
|                |       0|
|                |       0|
|                |       1|
|                |       1|
|                |       1|
|                |       1|
|                |       0|
|                |       0|
|                |       0|
|                |       0|
|                |       1|
|                |       1|
|                |       1|
|                |       1|
|                |       1|
|                |       1|
+----------------+--------+
only showing top 20 rows



In [10]:
Amazon_csv_ = Amazon_csv_.filter(Amazon_csv_.verified_reviews!=' ')

In [11]:
Amazon_csv_.show(truncate=True, n=10)

+--------------------+--------+
|    verified_reviews|feedback|
+--------------------+--------+
|       Love my Echo!|       1|
|           Loved it!|       1|
|Sometimes while p...|       1|
|I have had a lot ...|       1|
|               Music|       1|
|I received the ec...|       1|
|Without having a ...|       1|
|I think this is t...|       1|
|         looks great|       1|
|Love it! I’ve lis...|       1|
+--------------------+--------+
only showing top 10 rows



In [12]:
Amazon_csv_.filter(Amazon_csv.verified_reviews==' ').show()

+----------------+--------+
|verified_reviews|feedback|
+----------------+--------+
+----------------+--------+



In [13]:
Amazon_csv_.select('feedback').distinct().collect()

[Row(feedback='0'), Row(feedback='1')]

In [14]:
dividedData = Amazon_csv_.randomSplit([0.7, 0.3]) 
trainingData = dividedData[0]
testingData = dividedData[1] 
train_rows = trainingData.count()
test_rows = testingData.count()
print ("Training data rows:", train_rows, "; Testing data rows:", test_rows)

Training data rows: 2156 ; Testing data rows: 905


In [15]:
tokenizer = Tokenizer(inputCol="verified_reviews", outputCol="verified_reviews_words")
tokenizedTrain = tokenizer.transform(trainingData)
print(tokenizedTrain)

DataFrame[verified_reviews: string, feedback: string, verified_reviews_words: array<string>]


In [16]:
tokenizedTrain.printSchema()

root
 |-- verified_reviews: string (nullable = true)
 |-- feedback: string (nullable = true)
 |-- verified_reviews_words: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [17]:
tokenizedTrain.show(truncate=True, n=10)

+--------------------+--------+----------------------+
|    verified_reviews|feedback|verified_reviews_words|
+--------------------+--------+----------------------+
|"Handy if you don...|       1|  ["handy, if, you,...|
|"Handy if you don...|       1|  ["handy, if, you,...|
|"I love my echo p...|       1|  ["i, love, my, ec...|
|"I loved how easy...|       1|  ["i, loved, how, ...|
|"I purchased the ...|       1|  ["i, purchased, t...|
|"Love it except t...|       1|  ["love, it, excep...|
|"Overall love it....|       1|  ["overall, love, ...|
|&#34;NEVER BUY CE...|       0|  [&#34;never, buy,...|
|2nd one... more t...|       1|  [2nd, one..., mor...|
|   3rd Dot. Love it!|       1|  [3rd, dot., love,...|
+--------------------+--------+----------------------+
only showing top 10 rows



In [18]:
tokenizedTrain.select('feedback').distinct().collect()

[Row(feedback='0'), Row(feedback='1')]

In [19]:
tokenizedTrain.show()

+--------------------+--------+----------------------+
|    verified_reviews|feedback|verified_reviews_words|
+--------------------+--------+----------------------+
|"Handy if you don...|       1|  ["handy, if, you,...|
|"Handy if you don...|       1|  ["handy, if, you,...|
|"I love my echo p...|       1|  ["i, love, my, ec...|
|"I loved how easy...|       1|  ["i, loved, how, ...|
|"I purchased the ...|       1|  ["i, purchased, t...|
|"Love it except t...|       1|  ["love, it, excep...|
|"Overall love it....|       1|  ["overall, love, ...|
|&#34;NEVER BUY CE...|       0|  [&#34;never, buy,...|
|2nd one... more t...|       1|  [2nd, one..., mor...|
|   3rd Dot. Love it!|       1|  [3rd, dot., love,...|
|4 out of 5 stars....|       1|  [4, out, of, 5, s...|
|4.5 out of 5 Star...|       1|  [4.5, out, of, 5,...|
|4.5 out of 5 Star...|       1|  [4.5, out, of, 5,...|
|A GREAT PRODUCT.....|       1|  [a, great, produc...|
|         A great buy|       1|       [a, great, buy]|
|      A g

In [20]:
swr = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="MeaningfulWords")
SwRemovedTrain = swr.transform(tokenizedTrain)
SwRemovedTrain.show(truncate=True, n=5)

+--------------------+--------+----------------------+--------------------+
|    verified_reviews|feedback|verified_reviews_words|     MeaningfulWords|
+--------------------+--------+----------------------+--------------------+
|"Handy if you don...|       1|  ["handy, if, you,...|["handy, expect, ...|
|"Handy if you don...|       1|  ["handy, if, you,...|["handy, expect, ...|
|"I love my echo p...|       1|  ["i, love, my, ec...|["i, love, echo, ...|
|"I loved how easy...|       1|  ["i, loved, how, ...|["i, loved, easy,...|
|"I purchased the ...|       1|  ["i, purchased, t...|["i, purchased, "...|
+--------------------+--------+----------------------+--------------------+
only showing top 5 rows



In [21]:
hashTF = HashingTF(inputCol=swr.getOutputCol(), outputCol="features")
numericTrainData = hashTF.transform(SwRemovedTrain).select('feedback', 'MeaningfulWords', 'features')
numericTrainData.show(truncate=True, n=10)

+--------+--------------------+--------------------+
|feedback|     MeaningfulWords|            features|
+--------+--------------------+--------------------+
|       1|["handy, expect, ...|(262144,[30234,76...|
|       1|["handy, expect, ...|(262144,[30234,76...|
|       1|["i, love, echo, ...|(262144,[329,1900...|
|       1|["i, loved, easy,...|(262144,[36702,40...|
|       1|["i, purchased, "...|(262144,[769,1558...|
|       1|["love, except, a...|(262144,[4125,119...|
|       1|["overall, love, ...|(262144,[3502,880...|
|       0|[&#34;never, buy,...|(262144,[8970,430...|
|       1| [2nd, one..., come]|(262144,[114763,1...|
|       1|[3rd, dot., love,...|(262144,[83671,90...|
+--------+--------------------+--------------------+
only showing top 10 rows



In [22]:
numericTrainData.printSchema() 

root
 |-- feedback: string (nullable = true)
 |-- MeaningfulWords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)



In [23]:
numericTrainData.select('feedback').distinct().collect()

[Row(feedback='0'), Row(feedback='1')]

In [24]:
numericTrainData = numericTrainData.withColumn("feedback", 
                                  numericTrainData["feedback"]
                                  .cast('int'))

In [25]:
numericTrainData.printSchema() 

root
 |-- feedback: integer (nullable = true)
 |-- MeaningfulWords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)



In [26]:
lr = LogisticRegression(labelCol="feedback", featuresCol="features", maxIter=10, regParam=0.01)
model = lr.fit(numericTrainData)  
print ("Training is done!")

Training is done!


In [27]:
tokenizedTest = tokenizer.transform(testingData)
SwRemovedTest = swr.transform(tokenizedTest)
numericTest = hashTF.transform(SwRemovedTest).select('feedback', 'MeaningfulWords', 'features')
numericTest.show(truncate=True, n=10)

+--------+--------------------+--------------------+
|feedback|     MeaningfulWords|            features|
+--------+--------------------+--------------------+
|       1|["overall, love, ...|(262144,[3502,880...|
|       1|    [***love, it***]|(262144,[19700,20...|
|       1|[3rd, dot., love,...|(262144,[83671,90...|
|       1|[great, investmen...|(262144,[57422,13...|
|       1|[great, product.,...|(262144,[21310,17...|
|       1|[absolutely, love...|(262144,[18795,52...|
|       1|[absolutely, love...|(262144,[3960,167...|
|       1|[absolutely, love!!]|(262144,[102010,1...|
|       0|[couple, days, al...|(262144,[8538,168...|
|       1|[purchasing, echo...|(262144,[14974,16...|
+--------+--------------------+--------------------+
only showing top 10 rows



In [28]:
prediction = model.transform(numericTest)
predictionFinal = prediction.select("MeaningfulWords", "prediction", "feedback")
predictionFinal.show(n=10, truncate = True)

+--------------------+----------+--------+
|     MeaningfulWords|prediction|feedback|
+--------------------+----------+--------+
|["overall, love, ...|       1.0|       1|
|    [***love, it***]|       1.0|       1|
|[3rd, dot., love,...|       1.0|       1|
|[great, investmen...|       1.0|       1|
|[great, product.,...|       1.0|       1|
|[absolutely, love...|       1.0|       1|
|[absolutely, love...|       1.0|       1|
|[absolutely, love!!]|       1.0|       1|
|[couple, days, al...|       1.0|       0|
|[purchasing, echo...|       1.0|       1|
+--------------------+----------+--------+
only showing top 10 rows



In [29]:
correctPrediction = predictionFinal.filter(predictionFinal['prediction'] == predictionFinal['feedback']).count()
totalData = predictionFinal.count()
print("correct prediction:", correctPrediction, ", total data:", totalData, ", accuracy:", correctPrediction/totalData)

correct prediction: 842 , total data: 905 , accuracy: 0.9303867403314917


In [ ]:
strg = input("Enter a comment: ")
words = tokenize()